In [54]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://deleste1.educacao.sp.gov.br/lista-escolas-particulares/"

# Fazendo a requisição GET para obter o HTML
try:
    response = requests.get(url)
    response.raise_for_status()  # Verifica se houve erro na requisição
    html_content = response.text

except requests.exceptions.RequestException as e:
    print(f"Erro ao acessar a página: {e}")


# Parse o HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Inicializa uma lista para armazenar os dados
data = []

# Itera sobre todas as linhas da tabela (<tr>)
for row in soup.find_all('tr'):
    # Extrai os valores das células (<td>)
    cells = row.find_all('td')
    row_data = [cell.text.strip() for cell in cells]  # Remove espaços em branco
    data.append(row_data)

df = pd.DataFrame(data[1:], columns=data[0])


lista = df["ENDEREÇO"]

# Define a função lambda para apagar tudo após o "–"
apaga = lambda endereco: endereco.split("–")[0].strip()

# Aplica a função à coluna e cria uma nova coluna no DataFrame
df["ENDEREÇO_LIMPO"] = lista.map(apaga)
df['ENDEREÇO_LIMPO'] = df['ENDEREÇO_LIMPO'].apply(lambda x: f"{x}, São Paulo - SP")
df

,Nº,CIE,NOME DA ESCOLA,ENDEREÇO,FONE,TIPO DE ENSINO,ENDEREÇO_LIMPO
0,2,102155,"Adventista de Engº G, Escola","Rua Antônio R. de Almeida, 335 – Eng. Goulart",2623-1900,E.I./E.F. 9 anos do 1º ao 9º,"Rua Antônio R. de Almeida, 335, São Paulo - SP"
1,3,438170,"AFAM, Colégio","Rua Cel. Manuel Feliciano de Souza, 916 – Vila...",2037-7797,E.I./E.F. 9 anos/E.M.,"Rua Cel. Manuel Feliciano de Souza, 916, São P..."
2,4,114339,"Almeida Santos, Escola","Rua Américo Salvador Novelli, 256 – Itaquera",2745-2390,Profissionalizante – Enfermagem-Técnico,"Rua Américo Salvador Novelli, 256, São Paulo - SP"
3,5,163338,"Amorim, Colégio","Rua Rubens Galvão de França, 923 – Erm. Matarazzo",2546-3103/2943-0111,E.I./E.F. 9 anos do 1º ao 9º/E.M.,"Rua Rubens Galvão de França, 923, São Paulo - SP"
4,6,144952,"Argumento, Colégio – Unidade I","Av. João Batista de Oliveira, 350 – Erm. Matar...",2943-1002,E.F. 9 anos do 1º ao 9º/E.M.,"Av. João Batista de Oliveira, 350, São Paulo - SP"
...,...,...,...,...,...,...,...
87,90,006055,Tropical Escola,"Rua Miguel Rachid, 816 – Vila Paranaguá",2943-9905,E.I./E.F. do 1º ao 5º,"Rua Miguel Rachid, 816, São Paulo - SP"
88,91,157089,"União Paulista, Colégio","Rua Júlio Rinaldi, 230 – Vila Salete – Penha",2684-5429/2791-2011,E.I./E.F. 9 anos do 1º ao 9º,"Rua Júlio Rinaldi, 230, São Paulo - SP"
89,92,102301,"Universidade Brasil, Colégio","Rua Carolina Fonseca, 584 – Itaquera",2057-6008,E.I./E.F. 9 anos (1º ao 9º)/E.M./Téc.,"Rua Carolina Fonseca, 584, São Paulo - SP"
90,93,10374,Colégio Vila Real,"Rua Botuporã, 233 – Itaquera",11-94745-4847,E.F. 1º ao 5º,"Rua Botuporã, 233, São Paulo - SP"


In [73]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter as RL
import folium

# Inicializa o geolocalizador

df_enderecos = df['ENDEREÇO_LIMPO']
geolocator = Nominatim(user_agent="myGeocoder")
geocode = RL(geolocator.geocode, min_delay_seconds=1)

# Função para obter latitude e longitude a partir do endereço
def get_coordinates(address):
    try:
        #location = geolocator.geocode(address)
        location = geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erro ao localizar {address}: {e}")
        return None, None

# Adiciona colunas de latitude e longitude no DataFrame
df['Latitude'], df['Longitude'] = zip(*df_enderecos.apply(get_coordinates))

# Cria um mapa centralizado em São Paulo
mapa = folium.Map(location=[-23.55052, -46.633308], zoom_start=12)


# Adiciona marcadores para cada endereço no mapa
for _, row in df.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['ENDEREÇO_LIMPO']
        ).add_to(mapa)

# Exibe o mapa
mapa.save("mapa_enderecos.html")
print("Mapa salvo como 'mapa_enderecos.html'. Abra o arquivo no navegador.")

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rua Nª Senhora de Monte Vergini, 63, São Paulo - SP',), **{}).
Traceback (most recent call last):
  File "/home/wsantos/wsantos/lib/python3.12/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/home/wsantos/wsantos/lib/python3.12/site-packages/urllib3/connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 1428, in getresponse
    response.begin()
  File "/usr/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/pyth

Mapa salvo como 'mapa_enderecos.html'. Abra o arquivo no navegador.


In [80]:
df

,Nº,CIE,NOME DA ESCOLA,ENDEREÇO,FONE,TIPO DE ENSINO,ENDEREÇO_LIMPO,Latitude,Longitude
0,2,102155,"Adventista de Engº G, Escola","Rua Antônio R. de Almeida, 335 – Eng. Goulart",2623-1900,E.I./E.F. 9 anos do 1º ao 9º,"Rua Antônio R. de Almeida, 335, São Paulo - SP",-23.464835,-46.685545
1,3,438170,"AFAM, Colégio","Rua Cel. Manuel Feliciano de Souza, 916 – Vila...",2037-7797,E.I./E.F. 9 anos/E.M.,"Rua Cel. Manuel Feliciano de Souza, 916, São P...",-23.498346,-46.452942
2,4,114339,"Almeida Santos, Escola","Rua Américo Salvador Novelli, 256 – Itaquera",2745-2390,Profissionalizante – Enfermagem-Técnico,"Rua Américo Salvador Novelli, 256, São Paulo - SP",-23.537988,-46.454084
3,5,163338,"Amorim, Colégio","Rua Rubens Galvão de França, 923 – Erm. Matarazzo",2546-3103/2943-0111,E.I./E.F. 9 anos do 1º ao 9º/E.M.,"Rua Rubens Galvão de França, 923, São Paulo - SP",-23.502690,-46.485241
4,6,144952,"Argumento, Colégio – Unidade I","Av. João Batista de Oliveira, 350 – Erm. Matar...",2943-1002,E.F. 9 anos do 1º ao 9º/E.M.,"Av. João Batista de Oliveira, 350, São Paulo - SP",-23.500467,-46.469704
...,...,...,...,...,...,...,...,...,...
87,90,006055,Tropical Escola,"Rua Miguel Rachid, 816 – Vila Paranaguá",2943-9905,E.I./E.F. do 1º ao 5º,"Rua Miguel Rachid, 816, São Paulo - SP",-23.494571,-46.470182
88,91,157089,"União Paulista, Colégio","Rua Júlio Rinaldi, 230 – Vila Salete – Penha",2684-5429/2791-2011,E.I./E.F. 9 anos do 1º ao 9º,"Rua Júlio Rinaldi, 230, São Paulo - SP",-23.518751,-46.530487
89,92,102301,"Universidade Brasil, Colégio","Rua Carolina Fonseca, 584 – Itaquera",2057-6008,E.I./E.F. 9 anos (1º ao 9º)/E.M./Téc.,"Rua Carolina Fonseca, 584, São Paulo - SP",-23.528487,-46.455878
90,93,10374,Colégio Vila Real,"Rua Botuporã, 233 – Itaquera",11-94745-4847,E.F. 1º ao 5º,"Rua Botuporã, 233, São Paulo - SP",-23.542854,-46.446191


In [93]:
lista_nan = [i for i in df.index if pd.isna(df["Latitude"][i])]

print(lista_nan)
print(len(lista_nan))
print(len(df["Latitude"]))
print(f'{len(lista_nan)/len(df["Latitude"])*100} %')

[7, 13, 22, 23, 30, 31, 37, 42, 43, 63, 65, 70, 72, 79, 83]
15
92
16.304347826086957 %


In [112]:
print(df["ENDEREÇO_LIMPO"][65])
geolocator = Nominatim(user_agent="myGeocoder")
geocode = RL(geolocator.geocode, min_delay_seconds=1)
location = geocode("Rua Antônio de Castro Lopes, 613, SP").longitude
print(location)


Rua Padre Olivetanos, 783, São Paulo - SP
-46.93106765993117
